In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
from PTTLibrary import PTT

In [3]:
import json
def read_settings():
    with open('./settings.json') as file:
        setting = json.load(file)
    return setting

In [4]:
setting=read_settings()

In [5]:
PTTBot = PTT.Library()#LogLevel=PTT.LogLevel.DEBUG
try:
    PTTBot.login(
        ID=setting.get('ID'),
        Password=setting.get('Password'),
        KickOtherLogin=True
    )
except PTT.Exceptions.LoginError:
    PTTBot.log('登入失敗')
    sys.exit()
PTTBot.log('登入成功')

PTT Library v 0.8.13
Developed by PTT CodingMan
[0905 18:53:24][資訊] 繁體中文語言模組 [初始化]
[0905 18:53:24][資訊] 連線核心 [初始化]
[0905 18:53:24][資訊] 批踢踢函式庫 v 0.8.13 [初始化]
[0905 18:53:24][資訊] 登入帳號 [bigtongue]
[0905 18:53:24][資訊] 連線核心 [啟動]
[0905 18:53:24][資訊] 連線批踢踢 [WebSocket]
[0905 18:53:24][資訊] 批踢踢訊息 [登入中，請稍候]
[0905 18:53:24][資訊] 批踢踢訊息 [剔除其他登入]
[0905 18:53:29][資訊] 批踢踢訊息 [請按任意鍵繼續]
[0905 18:53:30][資訊] 批踢踢訊息 [登入成功]
[0905 18:53:30][資訊] 登入成功


In [6]:
Board='Stock'
SearchType=PTT.PostSearchType.Keyword
Condition='2019/04/16 盤中閒聊'
Index = PTTBot.getNewestIndex(
        PTT.IndexType.Board,
        Board,
        SearchType,
        SearchCondition=Condition,
    )
print(f'{Board} 最新文章編號 {Index}')
Post = PTTBot.getPost(
    Board,
    PostIndex=Index,
    SearchType=SearchType,
    SearchCondition=Condition,
)

print('標題:')
print(Post.getTitle())
print('內文:')
print(Post.getContent())
print('=' * 50)

Stock 最新文章編號 1
標題:
[閒聊] 2019/04/16 盤中閒聊
內文:
==============108/04/16台股資訊重點整理，供股民做投資參考=============
台  股  10875.6   ▲70.3(+0.65%)1187.65億
台指04 10876    ▲74  (+0.69%) 86,786口
盤後04 10862    ▼14  (-0.13%)  22,698口
摩台指 403.5      ▼0.2 (-0.05%)
八大公股             -0.32億 
台指期未平倉口數 自營商 -4,433(+1,306) 投信 -25,925(+228) 外資 +61,456(-1,652)
台指選未平倉口數 自營商 +14,391(+24,658) 投信 -2,273(-47) 外資 +99,928(+3,338)
P/C Ratio 129.21%  (+7.27%)
最大未平倉量                Call 10900 Put 10500
最大未平倉變化量  Call 10800 Put 10850
------------------------------------------------------------------
道瓊工業   26384.77 ▼27.53(-0.10%)
那斯達克    7976.01  ▼8.15(-0.10%)
標普500     2905.58 ▼1.83(-0.06%)
費城半導體 1485.82 ▼11.3(-0.75%)

擦鞋童發文，資料有若誤請多包涵

--


In [7]:
push_content_list = []
for push in Post.getPushList():
    push_content = push.getContent()
    push_content_list.append(push_content)
print(f'{len(Post.getPushList())}則推文')

1307則推文


## 預處理

In [8]:
import string
from zhon.hanzi import punctuation
push_content_list = [s.replace(' ', '') for s in push_content_list]
push_content_list = [s for s in push_content_list if s]
push_content_list = [''.join(c for c in s if c not in string.punctuation) for s in push_content_list]
push_content_list = [''.join(c for c in s if c not in punctuation) for s in push_content_list]

In [9]:
words_list = []
import jieba
jieba.load_userdict('user_dict.txt')
for push_content in push_content_list:
    words = jieba.lcut(push_content, cut_all=False)
    words_list = words_list + words

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BIGTON~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.538 seconds.
Prefix dict has been built succesfully.


## tf-idf

In [10]:
import os
from gensim import corpora, models, similarities

C:\Users\Bigtongue\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [11]:
# 載入語料庫
if (os.path.exists('ptt_stock_corpus.dict')):
    dictionary = corpora.Dictionary.load('ptt_stock_corpus.dict')
    corpus = corpora.MmCorpus('ptt_stock_corpus.mm') 
else:
    print('No corpus file')

C:\Users\Bigtongue\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [12]:
# 新資料
new_corpus = dictionary.doc2bow(words_list)
# 創建 tfidf model
tfidf = models.TfidfModel(corpus)
new_corpus_tfidf = tfidf[new_corpus]

In [13]:
new_corpus_tfidf.sort(key=lambda x: x[1],reverse=True)
for index,weight in new_corpus_tfidf[:10]:
    print(f'{dictionary[index]}:{weight}')

和碩:0.5851612174285397
3062:0.19308095062841366
錢線:0.1180880687857018
6116:0.11334565727801023
3491:0.10632906609842686
2929:0.1012183446734587
2643:0.09654047531420683
睿涵:0.09654047531420683
2313:0.09554611260448366
3338:0.08953317787000009
